## This notebook will work on cleaning the FIFA 21 data scraped from sofifa.com
Dataset is available at Kaggle Website.
Link: https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring

Dataset is Cleaned by: Puyush Gupta

In [1]:
# Importing Python Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

# We may get low_memory warning because guessing dtypes for each column is very memory demanding. 
# Pandas tries to determine what dtype to set by analyzing the data in each column. 
# So, We use the following synttax while loading dataset:
data = pd.read_csv("fifa21_raw_data_v2.csv", encoding='UTF-8', low_memory=False) 
# We have used encoding='UTF-8' as there are may special characters present in data like in columns 'Name', 'LongName', So, to replace those
# accented characters with plain English alternatives. 

# Taking the copy of Dataframe
df = data.copy()

# Let's see the basic information of dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

Extract information from Columns Descriptions File

In [2]:
content = []
with open('Fifa_Dataset_Dictionary.txt', 'r') as f:
    for line in f.readlines():
        content.append(line)
        
content = [i.strip('\n').split(': ') for i in content]

# Creating a Dictionary Format
content = {i[0]: i[1] for i in content} 

# Converting to dataframe
df2 = pd.DataFrame({'variable': content.keys(), 'description': content.values()}, index = np.arange(len(content)))

print(df2)
df2.to_csv('Column_Description_Fifa.csv', index=False)

       variable                                        description
0      photoUrl                     The URL of the player's photo.
1      LongName                       The full name of the player.
2     playerUrl        The URL of the player's page on sofifa.com.
3   Nationality                     The nationality of the player.
4     Positions                 The positions the player can play.
..          ...                                                ...
72          PAS  This refers to the player's passing ability. I...
73          DRI  This refers to the player's dribbling ability....
74          DEF  This refers to the player's defensive ability....
75          PHY  This refers to the player's physicality or str...
76         Hits  This refers to the number of times the player'...

[77 rows x 2 columns]


# Let's Start Cleaning Our Dataset

In [3]:
# To show each column with some defined row values to analysis better:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(2)

ID               Name                      LongName  \
0  158023           L. Messi                  Lionel Messi   
1   20801  Cristiano Ronaldo  C. Ronaldo dos Santos Aveiro   

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   

                                           playerUrl Nationality  Age  ↓OVA  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina   33    93   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal   35    92   

   POT                  Club     Contract   Positions Height Weight  \
0   93  \n\n\n\nFC Barcelona  2004 ~ 2021  RW, ST, CF  170cm   72kg   
1   92      \n\n\n\nJuventus  2018 ~ 2022      ST, LW  187cm   83kg   

  Preferred Foot  BOV Best Position        Joined Loan Date End    Value  \
0           Left   93            RW   Jul 1, 2004           NaN  €103.5M   
1          Right   92            ST  Jul 10, 2018           NaN     €63M   

    Wage Release Clause  Attacking  Crossing  Finishing  Heading Accuracy  \
0  €560K        €138.4M        429        85         95                70   
1  €220K         €75.9M        437        84         95                90   

   Short Passing  Volleys  Skill  Dribbling  Curve  FK Accuracy  Long Passing  \
0             91       88    470         96     93           94            91   
1             82       86    414         88     81           76            77   

   Ball Control  Movement  Acceleration  Sprint Speed  Agility  Reactions  \
0            96       451            91            80       91         94   
1            92       431            87            91       87         95   

   Balance  Power  Shot Power  Jumping  Stamina  Strength  Long Shots  \
0       95    389          86       68       72        69          94   
1       71    444          94       95       84        78          93   

   Mentality  Aggression  Interceptions  Positioning  Vision  Penalties  \
0        347          44             40           93      95         75   
1        353          63             29           95      82         84   

   Composure  Defending  Marking  Standing Tackle  Sliding Tackle  \
0         96         91       32               35              24   
1         95         84       28               32              24   

   Goalkeeping  GK Diving  GK Handling  GK Kicking  GK Positioning  \
0           54          6           11          15              14   
1           58          7           11          15              14   

   GK Reflexes  Total Stats  Base Stats  W/F  SM     A/W  D/W   IR  PAC  SHO  \
0            8         2231         466  4 ★  4★  Medium  Low  5 ★   85   92   
1           11         2221         464  4 ★  5★    High  Low  5 ★   89   93   

   PAS  DRI  DEF  PHY Hits  
0   91   95   38   65  771  
1   81   89   35   77  562

# Let's Check for duplicate rows in dataset

In [4]:
# Here, .duplicated() Return boolean Series denoting duplicate rows.
print(df[df.duplicated()])

# As There are no duplicates of rows, we will move ahead.

Empty DataFrame
Columns: [ID, Name, LongName, photoUrl, playerUrl, Nationality, Age, ↓OVA, POT, Club, Contract, Positions, Height, Weight, Preferred Foot, BOV, Best Position, Joined, Loan Date End, Value, Wage, Release Clause, Attacking, Crossing, Finishing, Heading Accuracy, Short Passing, Volleys, Skill, Dribbling, Curve, FK Accuracy, Long Passing, Ball Control, Movement, Acceleration, Sprint Speed, Agility, Reactions, Balance, Power, Shot Power, Jumping, Stamina, Strength, Long Shots, Mentality, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Defending, Marking, Standing Tackle, Sliding Tackle, Goalkeeping, GK Diving, GK Handling, GK Kicking, GK Positioning, GK Reflexes, Total Stats, Base Stats, W/F, SM, A/W, D/W, IR, PAC, SHO, PAS, DRI, DEF, PHY, Hits]
Index: []


# Let's find those columns which contain NAN value

In [5]:
# To get number of nan values in each column of dataframe: 

# Method: 1
df.isnull().sum()[df.isnull().sum() > 0]

# Method: 2 
# print(df.isna().sum().sort_values(ascending=False))

Loan Date End    17966
Hits              2595
dtype: int64

# Handling Missing Values

1. For Column 'Loan Date End'

In [6]:
# By domain knowledge and some Observation, I get to know: 

# In sports, a loan involves a particular player being able to temporarily play for a club other than the one to which they are currently contracted.
# Loaned players are not under full time contract and that particular column was created for only players who's contractual agreement is a loan. 
# So, then it means that those players with empty details at the loan end date column are players on full contract.
# I replaced all empty cells under the Loan End Date column with 0(indicating they are not loaned players).

# Unique values of 'Loan Date End
print(df['Loan Date End'].unique())

# To get the sample of dataset where 'Loan Date End' is not NAN
df.loc[df['Loan Date End'].notnull()].head(3)

[nan 'Jun 30, 2021' 'Dec 31, 2020' 'Jan 30, 2021' 'Jun 30, 2022'
 'May 31, 2021' 'Jul 5, 2021' 'Dec 31, 2021' 'Jul 1, 2021' 'Jan 1, 2021'
 'Aug 31, 2021' 'Jan 31, 2021' 'Dec 30, 2021' 'Jun 23, 2021' 'Jan 3, 2021'
 'Nov 27, 2021' 'Jan 17, 2021' 'Jun 30, 2023' 'Jul 31, 2021'
 'Nov 22, 2020' 'May 31, 2022' 'Dec 30, 2020' 'Jan 4, 2021' 'Nov 30, 2020'
 'Aug 1, 2021']


ID            Name                   LongName  \
205  173731         G. Bale                Gareth Bale   
248  193105       A. Areola            Alphonse Areola   
254  200888  Danilo Pereira  Danilo Luís Hélio Pereira   

                                             photoUrl  \
205  https://cdn.sofifa.com/players/173/731/21_60.png   
248  https://cdn.sofifa.com/players/193/105/21_60.png   
254  https://cdn.sofifa.com/players/200/888/21_60.png   

                                             playerUrl Nationality  Age  ↓OVA  \
205  http://sofifa.com/player/173731/gareth-bale/21...       Wales   30    83   
248  http://sofifa.com/player/193105/alphonse-areol...      France   27    82   
254  http://sofifa.com/player/200888/danilo-luis-he...    Portugal   28    82   

     POT                         Club              Contract Positions Height  \
205   83    \n\n\n\nTottenham Hotspur  Jun 30, 2021 On Loan        RW  185cm   
248   86               \n\n\n\nFulham  Jun 30, 2021 On Loan        GK  195cm   
254   82  \n\n\n\nParis Saint-Germain  Jun 30, 2021 On Loan   CDM, CM  188cm   

    Weight Preferred Foot  BOV Best Position       Joined Loan Date End Value  \
205   82kg           Left   83            RW  Sep 1, 2013  Jun 30, 2021  €35M   
248   94kg          Right   82            GK  Jul 1, 2010  Jun 30, 2021  €32M   
254   83kg          Right   82           CDM  Jul 2, 2015  Jun 30, 2021  €27M   

      Wage Release Clause  Attacking  Crossing  Finishing  Heading Accuracy  \
205  €200K             €0        417        87         81                84   
248   €71K             €0        117        20         19                14   
254   €20K             €0        329        58         57                76   

     Short Passing  Volleys  Skill  Dribbling  Curve  FK Accuracy  \
205             80       85    419         80     90           87   
248             48       16    103         12     16           16   
254             83       55    340         75     54           53   

     Long Passing  Ball Control  Movement  Acceleration  Sprint Speed  \
205            79            83       397            86            89   
248            37            22       305            56            54   
254            80            78       323            63            68   

     Agility  Reactions  Balance  Power  Shot Power  Jumping  Stamina  \
205       78         79       65    405          90       82       69   
248       58         79       58    264          60       72       38   
254       57         75       60    377          74       62       84   

     Strength  Long Shots  Mentality  Aggression  Interceptions  Positioning  \
205        77          87        362          65             59           83   
248        80          14        142          26             23           17   
254        89          68        363          82             81           71   

     Vision  Penalties  Composure  Defending  Marking  Standing Tackle  \
205      79         76         85        161       54               55   
248      51         25         64         43       13               18   
254      72         57         84        234       83               83   

     Sliding Tackle  Goalkeeping  GK Diving  GK Handling  GK Kicking  \
205              52           52         15           15          11   
248              12          406         85           79          80   
254              68           58          9            7          13   

     GK Positioning  GK Reflexes  Total Stats  Base Stats  W/F  SM     A/W  \
205               5            6         2213         465  3 ★  4★  Medium   
248              78           84         1380         461  3 ★  1★  Medium   
254              14           15         2024         439  3 ★  3★  Medium   

        D/W   IR  PAC  SHO  PAS  DRI  DEF  PHY Hits  
205  Medium  4 ★   88   84   82   80   58   73  282  
248  Medium  2 ★   85   79   80   84   55   78   94  
254    High  3

# Strategy:
Our 'Loan Date End' is really similar to the 'Contract' column. So, We will be looking towards creating a new column which we will call the "Contract Expiry Year" as We already have 'Joined' column which contains information when a player joined the club.

This new column would consist of the End year in the Contract i.e. the year when the contract of player will be over. We would also be dropping 'Loan Date End' column afterwards since it is not really needed then. 

We will change the column name 'Contract' to 'Player Status', converting it to a categorical column. Also, to preserve the information, that which player is on Loan or not or are Free, We will replace values containing 'Free' in ''Player Status' column with 'Free'; values containing 'Loan' in 'Player Status' column with 'Loan' and values containing '~' in 'Player Status' column with 'Contract'.

In [7]:
# Creating the 2 columns 'Month' and 'Contract Expiry Year' from 'Loan Date End' Values
df[['Month','Contract Expiry Year']] = df['Loan Date End'].str.split(',', expand = True).apply(lambda x: x.str.strip())
# We are using .str as dtype of 'Loan Date End' is 'object' originally

# We keep the last 4 characters of each value from our 'Contract' column and use it to fill the NAN values in our 'Contract Expiry Year' 

expiry_yr = df['Contract'].apply(lambda x : x[-4:])
df['Contract Expiry Year'] = df['Contract Expiry Year'].fillna(expiry_yr)

# Now rename 'Contract' to a categorical column 'Player Status' for players with permanent Contract, Loan Contract and those who are Free
df.rename(columns = {'Contract': 'Player Status'}, inplace = True)

# Function create for column 'Player Status',
def status(val):
    # Assign 'Free' to Free players with no contract
    if 'Free' in val:
        return 'Free'
    # Assign 'Loan' to Loan players with temporary club
    elif 'Loan' in val:
        return 'Loan'
    # Assign 'Contract' to players with contract with parent club
    elif '~' in val:
        return 'Contract'
    
df['Player Status'] = df['Player Status'].apply(status)

# About categorical data:
# When your variable has only a few possible values (df[var].nunique() to work out the number of different possible values) it would be a good idea to set it as a categorical variable ---> df['var'].astype(category)
df['Player Status'] = df['Player Status'].astype('category')


print(df['Player Status'].unique())
print(df['Player Status'].isna().sum()) #As, it's 0, We are done.
df.head(3)

['Contract', 'Loan', 'Free']
Categories (3, object): ['Contract', 'Free', 'Loan']
0


ID               Name                      LongName  \
0  158023           L. Messi                  Lionel Messi   
1   20801  Cristiano Ronaldo  C. Ronaldo dos Santos Aveiro   
2  200389           J. Oblak                     Jan Oblak   

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   
2  https://cdn.sofifa.com/players/200/389/21_60.png   

                                           playerUrl Nationality  Age  ↓OVA  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina   33    93   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal   35    92   
2  http://sofifa.com/player/200389/jan-oblak/210006/    Slovenia   27    91   

   POT                     Club Player Status   Positions Height Weight  \
0   93     \n\n\n\nFC Barcelona      Contract  RW, ST, CF  170cm   72kg   
1   92         \n\n\n\nJuventus      Contract      ST, LW  187cm   83kg   
2   93  \n\n\n\nAtlético Madrid      Contract          GK  188cm   87kg   

  Preferred Foot  BOV Best Position        Joined Loan Date End    Value  \
0           Left   93            RW   Jul 1, 2004           NaN  €103.5M   
1          Right   92            ST  Jul 10, 2018           NaN     €63M   
2          Right   91            GK  Jul 16, 2014           NaN    €120M   

    Wage Release Clause  Attacking  Crossing  Finishing  Heading Accuracy  \
0  €560K        €138.4M        429        85         95                70   
1  €220K         €75.9M        437        84         95                90   
2  €125K        €159.4M         95        13         11                15   

   Short Passing  Volleys  Skill  Dribbling  Curve  FK Accuracy  Long Passing  \
0             91       88    470         96     93           94            91   
1             82       86    414         88     81           76            77   
2             43       13    109         12     13           14            40   

   Ball Control  Movement  Acceleration  Sprint Speed  Agility  Reactions  \
0            96       451            91            80       91         94   
1            92       431            87            91       87         95   
2            30       307            43            60       67         88   

   Balance  Power  Shot Power  Jumping  Stamina  Strength  Long Shots  \
0       95    389          86       68       72        69          94   
1       71    444          94       95       84        78          93   
2       49    268          59       78       41        78          12   

   Mentality  Aggression  Interceptions  Positioning  Vision  Penalties  \
0        347          44             40           93      95         75   
1        353          63             29           95      82         84   
2        140          34             19           11      65         11   

   Composure  Defending  Marking  Standing Tackle  Sliding Tackle  \
0         96         91       32               35              24   
1         95         84       28               32              24   
2         68         57       27               12              18   

   Goalkeeping  GK Diving  GK Handling  GK Kicking  GK Positioning  \
0           54          6           11          15              14   
1           58          7           11          15              14   
2          437         87           92          78              90   

   GK Reflexes  Total Stats  Base Stats  W/F  SM     A/W     D/W   IR  PAC  \
0            8         2231         466  4 ★  4★  Medium     Low  5 ★   85   
1           11         2221         464  4 ★  5★    High     Low  5 ★   89   
2           90         1413         489  3 ★  1★  Medium  Medium  3 ★   87   

   SHO  PAS  DRI  DEF  PHY Hits Month Contract Expiry Year  
0   92   91   95   38   65  771   NaN                 2021  
1   93   81   89   35   77  562   NaN                 2022  
2   92   78   90   52  

In [8]:
# Dropping Irrelevant Columns:

# Dropping 'Name' too as We have 'LongName' Column already
drop_columns = ['Month', 'Loan Date End' , 'Name']
df.drop(columns = drop_columns, inplace=True)   

# Changing column name of 'LongName' to 'Full Name'
df.rename(columns = {'LongName': 'Full_Name'}, inplace = True)

2. For Column 'Hits'

In [9]:
# 'Hits' can be cleaned and transformed into integer (later converted to ordered categorical)
# Hits(means the number of times a player profile has been viewed)- we can set this to 0 meaning the player has no view. 
# We can also fill it with mean, it depends on situation.
print(df['Hits'].unique())

# To handle values like 1.6k etc.
def clean_hits(i):
    if i[-1] == 'K':
        i = i[:-1]
        i = float(i) * 1000
        i = int(i)
    else:
        i = int(i)
    return i


df['Hits'] = df['Hits'].fillna(0)
df['Hits'] = df['Hits'].astype(str).apply(clean_hits)

print(df['Hits'].isnull().sum())
# As, it's 0, We have handled all the nan-values and other too.

['771' '562' '150' '207' '595' '248' '246' '120' '1.6K' '130' '321' '189'
 '175' '96' '118' '216' '212' '154' '205' '202' '339' '408' '103' '332'
 '86' '173' '161' '396' '1.1K' '433' '242' '206' '177' '1.5K' '198' '459'
 '117' '119' '209' '84' '187' '165' '203' '65' '336' '126' '313' '124'
 '145' '538' '182' '101' '45' '377' '99' '194' '403' '414' '593' '374'
 '245' '3.2K' '266' '299' '309' '215' '265' '211' '112' '337' '70' '159'
 '688' '116' '63' '144' '123' '71' '224' '113' '168' '61' '89' '137' '278'
 '75' '148' '176' '197' '264' '214' '247' '402' '440' '1.7K' '2.3K' '171'
 '320' '657' '87' '259' '200' '255' '253' '196' '60' '97' '85' '169' '256'
 '132' '239' '166' '121' '109' '32' '46' '122' '48' '527' '199' '282' '51'
 '1.9K' '642' '155' '323' '288' '497' '509' '79' '49' '270' '511' '80'
 '128' '115' '156' '204' '143' '140' '152' '220' '134' '225' '94' '74'
 '135' '142' '50' '77' '40' '107' '193' '179' '34' '64' '453' '57' '81'
 '28' '78' '133' '43' '425' '88' '42' '36' '233' '37

# Appropriate Actions:

Some conclusions after observing the data columns:  
1. Rename '↓OVA' column to 'OVA'.
2. 'Club' needs some cleaning. Remove the leading '\n'.
3. 'Height' values could be transformed to 'meters' as it having some values in 'cm' and some in 'foot' and converting the dtype to float.
4. 'Weight' values could be transformed to 'kg' as it having some values in 'kg' and some in 'lbs' and converting the dtype to float.
5. Renaming 'Preferred Foot' to 'Foot', 'Best Position' to 'Position' and they definitely must be converted to categorical variables.
6. 'Joined' must be datetime dtype. Also, It will be appropriate if We set this column as index while sorting the dataframe according to the date
7. 'Wage', 'Value' & 'Release Clause' can be converted to float. Remove euro sign and converting all values in scale of million.
8. 'A/W' and 'D/W' can be converted to ordered category and it has very few value.
9. 'W/F', 'SM' and 'IR' must be converted to integer and then to ordered category. Remove the star at end of each value.

1. For Column '↓OVA'

In [10]:
df.rename(columns = {'↓OVA':'OVA'}, inplace = True)

2. For Column 'Club'

In [11]:
# Method 1:
# The strip() function assists in removing character \n from the beginning or end of a string
df['Club'] = df['Club'].astype(str).apply(lambda x: x.strip('\n'))

# Method 2:
# data['Club'] = data['Club'].str.replace('\n\n\n\n', '')

df.head(2)

ID                     Full_Name  \
0  158023                  Lionel Messi   
1   20801  C. Ronaldo dos Santos Aveiro   

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   

                                           playerUrl Nationality  Age  OVA  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina   33   93   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal   35   92   

   POT          Club Player Status   Positions Height Weight Preferred Foot  \
0   93  FC Barcelona      Contract  RW, ST, CF  170cm   72kg           Left   
1   92      Juventus      Contract      ST, LW  187cm   83kg          Right   

   BOV Best Position        Joined    Value   Wage Release Clause  Attacking  \
0   93            RW   Jul 1, 2004  €103.5M  €560K        €138.4M        429   
1   92            ST  Jul 10, 2018     €63M  €220K         €75.9M        437   

   Crossing  Finishing  Heading Accuracy  Short Passing  Volleys  Skill  \
0        85         95                70             91       88    470   
1        84         95                90             82       86    414   

   Dribbling  Curve  FK Accuracy  Long Passing  Ball Control  Movement  \
0         96     93           94            91            96       451   
1         88     81           76            77            92       431   

   Acceleration  Sprint Speed  Agility  Reactions  Balance  Power  Shot Power  \
0            91            80       91         94       95    389          86   
1            87            91       87         95       71    444          94   

   Jumping  Stamina  Strength  Long Shots  Mentality  Aggression  \
0       68       72        69          94        347          44   
1       95       84        78          93        353          63   

   Interceptions  Positioning  Vision  Penalties  Composure  Defending  \
0             40           93      95         75         96         91   
1             29           95      82         84         95         84   

   Marking  Standing Tackle  Sliding Tackle  Goalkeeping  GK Diving  \
0       32               35              24           54          6   
1       28               32              24           58          7   

   GK Handling  GK Kicking  GK Positioning  GK Reflexes  Total Stats  \
0           11          15              14            8         2231   
1           11          15              14           11         2221   

   Base Stats  W/F  SM     A/W  D/W   IR  PAC  SHO  PAS  DRI  DEF  PHY  Hits  \
0         466  4 ★  4★  Medium  Low  5 ★   85   92   91   95   38   65   771   
1         464  4 ★  5★    High  Low  5 ★   89   93   81   89   35   77   562   

  Contract Expiry Year  
0                 2021  
1                 2022

3. For Column 'Height'

In [12]:
print(df['Height'].unique())
# As, 1 inch = 2.54 cm and 1 foot = 12 inch and 1 foot = 30.48 cm

def convert_height(x):
    if x[-1] == "\"":
        x = x.replace('"', "") # or use x.replace("/"", "") 
        foot = int(x[0]) * 30.48
        inch = int(x[2:]) * 2.54
        return (foot+inch)/100
    elif x[-1] == "m":
        return int(x[:-2])/100

df.Height = df["Height"].apply(convert_height)

df.rename(columns = {'Height': 'Height(m)'}, inplace = True)

['170cm' '187cm' '188cm' '181cm' '175cm' '184cm' '191cm' '178cm' '193cm'
 '185cm' '199cm' '173cm' '168cm' '176cm' '177cm' '183cm' '180cm' '189cm'
 '179cm' '195cm' '172cm' '182cm' '186cm' '192cm' '165cm' '194cm' '167cm'
 '196cm' '163cm' '190cm' '174cm' '169cm' '171cm' '197cm' '200cm' '166cm'
 '6\'2"' '164cm' '198cm' '6\'3"' '6\'5"' '5\'11"' '6\'4"' '6\'1"' '6\'0"'
 '5\'10"' '5\'9"' '5\'6"' '5\'7"' '5\'4"' '201cm' '158cm' '162cm' '161cm'
 '160cm' '203cm' '157cm' '156cm' '202cm' '159cm' '206cm' '155cm']


4. For Column 'Weight'

In [13]:
print(df['Weight'].unique())

#As, 1 lbs = 0.453592 kg

def convert_weight(x):
    if x[-3:] == "lbs":
        return round(int(x[:-3])*0.4536, 2)
    elif x[-2:] == "kg":
        return int(x[:-2])
      
    
df.Weight = df["Weight"].apply(convert_weight)
df.rename(columns = {'Weight': 'Weight(Kg)'}, inplace = True)

['72kg' '83kg' '87kg' '70kg' '68kg' '80kg' '71kg' '91kg' '73kg' '85kg'
 '92kg' '69kg' '84kg' '96kg' '81kg' '82kg' '75kg' '86kg' '89kg' '74kg'
 '76kg' '64kg' '78kg' '90kg' '66kg' '60kg' '94kg' '79kg' '67kg' '65kg'
 '59kg' '61kg' '93kg' '88kg' '97kg' '77kg' '62kg' '63kg' '95kg' '100kg'
 '58kg' '183lbs' '179lbs' '172lbs' '196lbs' '176lbs' '185lbs' '170lbs'
 '203lbs' '168lbs' '161lbs' '146lbs' '130lbs' '190lbs' '174lbs' '148lbs'
 '165lbs' '159lbs' '192lbs' '181lbs' '139lbs' '154lbs' '157lbs' '163lbs'
 '98kg' '103kg' '99kg' '102kg' '56kg' '101kg' '57kg' '55kg' '104kg'
 '107kg' '110kg' '53kg' '50kg' '54kg' '52kg']


5. For Columns 'Preferred Foot' and 'Best Position'

In [14]:
# For 'Preferred Foot':
print(df['Preferred Foot'].unique())

# Foot must be converted to Categorical
df['Preferred Foot'] = df['Preferred Foot'].astype('category')

print(df.dtypes['Preferred Foot'])


# For 'Best Position':
print(df['Best Position'].unique())

# Position must be converted to Categorical
df['Best Position'] = df['Best Position'].astype('category')

print(df.dtypes['Best Position'])

['Left' 'Right']
category
['RW' 'ST' 'GK' 'CAM' 'LW' 'CB' 'CDM' 'CF' 'CM' 'RB' 'LB' 'LM' 'RM' 'LWB'
 'RWB']
category


6. For Column 'Joined'

In [15]:
# We have to convert the date to yyyy/mm/dd format which is in a datetime format
print(df['Joined'].unique())
print(df['Joined'].dtype)

joined = df.Joined

print(pd.to_datetime(joined))

print(pd.to_datetime(joined).isna().sum())   # As, it return 0, So, Everything is good, so let's assign the value

df.Joined = pd.to_datetime(joined)

# Setting 'Joined' as index
df.set_index('Joined', inplace=True)
# Sorting Dataframe according to Date i.e. when players joined the club
df = df.sort_index(ascending=True)

df.head(2)

['Jul 1, 2004' 'Jul 10, 2018' 'Jul 16, 2014' ... 'Sep 22, 2018'
 'Feb 28, 2015' 'Mar 6, 2018']
object
0       2004-07-01
1       2018-07-10
2       2014-07-16
3       2015-08-30
4       2017-08-03
5       2014-07-01
6       2017-07-01
7       2018-07-19
8       2018-07-01
9       2014-07-01
10      2018-01-01
11      2016-07-01
12      2013-07-11
13      2018-08-09
14      2011-07-01
15      2009-07-09
16      2005-08-01
17      2011-07-28
18      2015-07-14
19      2016-07-16
20      2015-07-01
21      2015-07-01
22      2017-07-01
23      2010-07-01
24      2012-07-01
25      2014-07-01
26      2019-07-01
27      2014-07-17
28      2019-07-12
29      2017-08-31
30      2015-10-07
31      2017-07-01
32      2017-07-21
33      2018-01-30
34      2020-01-30
35      2018-07-01
36      2015-08-28
37      2015-07-06
38      2019-09-02
39      2005-07-01
40      2008-09-01
41      2018-01-31
42      2017-07-19
43      2015-08-06
44      2012-08-01
45      2020-09-25
46      2012-08-31
47   

ID         Full_Name  \
Joined                                 
1998-01-01  140181  Hitoshi Sogahata   
2002-01-01  184900     Kwang Suk Kim   

                                                    photoUrl  \
Joined                                                         
1998-01-01  https://cdn.sofifa.com/players/140/181/21_60.png   
2002-01-01  https://cdn.sofifa.com/players/184/900/21_60.png   

                                                    playerUrl     Nationality  \
Joined                                                                          
1998-01-01  http://sofifa.com/player/140181/hitoshi-sogaha...           Japan   
2002-01-01  http://sofifa.com/player/184900/kwang-suk-kim/...  Korea Republic   

            Age  OVA  POT             Club Player Status Positions  Height(m)  \
Joined                                                                          
1998-01-01   40   65   65  Kashima Antlers      Contract        GK       1.87   
2002-01-01   37   70   70  Pohang Steelers      Contract        CB       1.83   

            Weight(Kg) Preferred Foot  BOV Best Position  Value Wage  \
Joined                                                                 
1998-01-01        80.0          Right   65            GK   €80K  €1K   
2002-01-01        73.0          Right   70            CB  €275K  €3K   

           Release Clause  Attacking  Crossing  Finishing  Heading Accuracy  \
Joined                                                                        
1998-01-01           €63K         87        13         12                18   
2002-01-01          €438K        260        58         38                70   

            Short Passing  Volleys  Skill  Dribbling  Curve  FK Accuracy  \
Joined                                                                     
1998-01-01             26       18     84         14     10           11   
2002-01-01             55       39    232         51     46           30   

            Long Passing  Ball Control  Movement  Acceleration  Sprint Speed  \
Joined                                                                         
1998-01-01            33            16       143            17            19   
2002-01-01            49            56       325            69            61   

            Agility  Reactions  Balance  Power  Shot Power  Jumping  Stamina  \
Joined                                                                         
1998-01-01       28         47       32    203          48       52       37   
2002-01-01       54         73       68    316          48       89       63   

            Strength  Long Shots  Mentality  Aggression  Interceptions  \
Joined                                                                   
1998-01-01        51          15        100          21             12   
2002-01-01        75          41        285          71             73   

            Positioning  Vision  Penalties  Composure  Defending  Marking  \
Joined                                                                      
1998-01-01           10      45         12         20         42       17   
2002-01-01           55      43         43         56        205       70   

            Standing Tackle  Sliding Tackle  Goalkeeping  GK Diving  \
Joined                                                                
1998-01-01               12              13          331         64   
2002-01-01               69              66           53          9   

            GK Handling  GK Kicking  GK Positioning  GK Reflexes  Total Stats  \
Joined                                                                          
1998-01-01           66          64              72           65          990   
2002-01-01            8          16              13            7         1676   

            Base Stats  W/F  SM     A/W     D/W   IR  PAC  SHO  PAS  DRI  DEF  \
Joined                                                                          
1998-01-01       

7. For Columns 'Wage', 'Value' and 'Release Clause'

In [16]:
# print(df['Wage'].unique())
print(df['Wage'].dtype)

# print(df['Value'].unique())
print(df['Value'].dtype)

# print(df['Release Clause'].unique())
print(df['Release Clause'].dtype)

value = df.Value
wage = df.Wage
release_clause = df['Release Clause']

# Let's check if all the values are in Euro

print(value.str[0].unique())
print(wage.str[0].unique())
print(release_clause.str[0].unique())

# Let's check if all the values are in Million/Thousand

print(value.str[-1].unique())
print(wage.str[-1].unique())
print(release_clause.str[-1].unique())

object
object
object
['€']
['€']
['€']
['K' 'M' '0']
['K' '0']
['K' 'M' '0']


In [17]:
def to_millions(i):
    if i[-1] == 'M':
        value = float(i[:-1])
        return '{}'.format(int(value))
    elif i[-1] == 'K':
        value = float(i[:-1])/1000
        return '{}'.format(value)
    else:
        value = float(i)/1000000
        return '{}'.format(value)  


# Removing Euro Signs
value = value.str[1:]
wage = wage.str[1:]
release_clause = release_clause.str[1:]

value = value.apply(to_millions)
wage = wage.apply(to_millions)
release_clause = release_clause.apply(to_millions)

df.Value = value
df.Wage = wage
df['Release Clause'] = release_clause

df['Value'] = df['Value'].astype(float)
df['Wage'] = df['Wage'].astype(float)
df['Release Clause'] = df['Release Clause'].astype(float)

df.rename(columns = {'Value' : 'Value(Euro In Millions)','Wage': 'Wage(Euro In Millions)','Release Clause': 'Release Clause(Euro In Millions)'},inplace='True')

8. For Columns 'A/W' and 'D/W'

In [18]:
print(df['A/W'].unique())

print(df['D/W'].unique())

df['A/W'] = df['A/W'].astype('category')
df['D/W'] = df['D/W'].astype('category')

df['A/W'] = df['A/W'].cat.set_categories(['Low', 'Medium', 'High'], ordered=True)
df['D/W'] = df['D/W'].cat.set_categories(['Low', 'Medium', 'High'], ordered=True)

['Medium' 'Low' 'High']
['Medium' 'High' 'Low']


9. For Columns 'W/F', 'SM' and 'IR'

In [19]:
print(df['W/F'].dtype)
print(df['W/F'].unique())

print(df['SM'].dtype)
print(df['SM'].unique())


print(df['IR'].dtype)
print(df['IR'].unique())

def remove_star(i):
    return int(i[0])


df['W/F'] = df['W/F'].apply(remove_star)
df['SM'] = df['SM'].apply(remove_star)
df['IR'] = df['IR'].apply(remove_star)

df['W/F'] = df['W/F'].astype('category')
df['SM'] = df['SM'].astype('category')
df['IR'] = df['IR'].astype('category')

# You can convert an unordered categorical type column to an ordered categorical column.
df['W/F'] = df['W/F'].cat.set_categories([1,2,3,4,5], ordered=True)
df['SM'] = df['SM'].cat.set_categories([1,2,3,4,5], ordered=True)
df['IR'] = df['IR'].cat.set_categories([1,2,3,4,5], ordered=True)

df = df.rename(columns={'W/F':'W/F(★)','SM':'SM(★)','IR':'IR(★)'})

object
['3 ★' '2 ★' '4 ★' '1 ★' '5 ★']
object
['1★' '2★' '3★' '4★' '5★']
object
['1 ★' '3 ★' '2 ★' '5 ★' '4 ★']


In [20]:
df.sample(5)

ID             Full_Name  \
Joined                                     
2014-12-11  192020       Emilio Orrantía   
2018-08-09  206202         Ollie Norburn   
2017-06-23  178283           Ruud Boffin   
2020-09-02  255634  Georgios Vagiannidis   
2019-01-01  255879      Martín Fernández   

                                                    photoUrl  \
Joined                                                         
2014-12-11  https://cdn.sofifa.com/players/192/020/21_60.png   
2018-08-09  https://cdn.sofifa.com/players/206/202/21_60.png   
2017-06-23  https://cdn.sofifa.com/players/178/283/21_60.png   
2020-09-02  https://cdn.sofifa.com/players/255/634/21_60.png   
2019-01-01  https://cdn.sofifa.com/players/255/879/21_60.png   

                                                    playerUrl Nationality  \
Joined                                                                      
2014-12-11  http://sofifa.com/player/192020/emilio-orranti...      Mexico   
2018-08-09  http://sofifa.com/player/206202/ollie-norburn/...     Grenada   
2017-06-23  http://sofifa.com/player/178283/ruud-boffin/21...     Belgium   
2020-09-02  http://sofifa.com/player/255634/georgios-vagia...      Greece   
2019-01-01  http://sofifa.com/player/255879/martin-fernand...     Uruguay   

            Age  OVA  POT                   Club Player Status Positions  \
Joined                                                                     
2014-12-11   29   65   65          Santos Laguna      Contract    RB, RM   
2018-08-09   27   64   65             Shrewsbury      Contract        CM   
2017-06-23   32   70   70            Antalyaspor      Contract        GK   
2020-09-02   18   57   72      Sint-Truidense VV      Contract        RB   
2019-01-01   19   60   72  Liverpool Fútbol Club      Contract        CM   

            Height(m)  Weight(Kg) Preferred Foot  BOV Best Position  \
Joined                                                                
2014-12-11       1.76        63.0          Right   65            RB   
2018-08-09       1.85        82.0          Right   66            RM   
2017-06-23       1.96        86.0          Right   70            GK   
2020-09-02       1.82        68.0          Right   57            RB   
2019-01-01       1.83        75.0          Right   62            CM   

            Value(Euro In Millions)  Wage(Euro In Millions)  \
Joined                                                        
2014-12-11                    0.625                 0.00600   
2018-08-09                    0.675                 0.00300   
2017-06-23                    0.800                 0.00800   
2020-09-02                    0.375                 0.00065   
2019-01-01                    0.575                 0.00050   

            Release Clause(Euro In Millions)  Attacking  Crossing  Finishing  \
Joined                                                                         
2014-12-11                             0.765        271        66         46   
2018-08-09                             0.913        292        62         58   
2017-06-23                             2.000         71        11         13   
2020-09-02                             0.415        222        48         41   
2019-01-01                             0.823        237        36         55   

            Heading Accuracy  Short Passing  Volleys  Skill  Dribbling  Curve  \
Joined                                                                          
2014-12-11                48             64       47    303         65     56   
2018-08-09                55             66       51    315         65     61   
2017-06-23                12             18       17     80         20     17   
2020-09-02                48             47       38    206         45     42   
2019-01-01                47             66       33    253         52     37   

            FK Accuracy  Long Passing  Ball Control  Movement  Acceleration  \
Joined                               

# Note that:

Attacking = sum(Crossing, Finishing, Heading Accuracy, Short Passing, Volleys)

Skill = sum(Dribbling, Curve, FK Accuracy, Long Passing, Ball Control) 

Movement = sum(Acceleration, Sprint Speed, Agility, Reactions, Balance)

Power = sum(Shot Power, Jumping, Stamina, Strength, Long Shots)

Mentality = sum(Agression, Interceptions, Positioning, Vision, Penalties, Composure)

Defending = sum(Marking, Standing Tackle, Sliding Tackle)

GoalKeeping = sum(GK Diving, GK Handling, GK Kicking, GK Positioning, GK Reflexes)

Total stats = sum(Attacking, Skill, Movement, Power, Mentality, Defending, GoalKeeping)

Base Stats = sum (Pace, Shooting, Passing, Dribbling, Defending, and Physicality. ie. (PAC,	SHO, PAS, DRI, DEF, PHY	))


# Here, 'Composure' is not added to 'Mentality' and 'Totat Stats'. 

In [22]:
# Let's add Composure to 'Mentality' and 'Total Stats' columns.

df['Mentality'] = df['Mentality'] + df['Composure']
df['Total Stats'] = df['Total Stats'] + df['Composure']

In [23]:
df.head(2)

ID         Full_Name  \
Joined                                 
1998-01-01  140181  Hitoshi Sogahata   
2002-01-01  184900     Kwang Suk Kim   

                                                    photoUrl  \
Joined                                                         
1998-01-01  https://cdn.sofifa.com/players/140/181/21_60.png   
2002-01-01  https://cdn.sofifa.com/players/184/900/21_60.png   

                                                    playerUrl     Nationality  \
Joined                                                                          
1998-01-01  http://sofifa.com/player/140181/hitoshi-sogaha...           Japan   
2002-01-01  http://sofifa.com/player/184900/kwang-suk-kim/...  Korea Republic   

            Age  OVA  POT             Club Player Status Positions  Height(m)  \
Joined                                                                          
1998-01-01   40   65   65  Kashima Antlers      Contract        GK       1.87   
2002-01-01   37   70   70  Pohang Steelers      Contract        CB       1.83   

            Weight(Kg) Preferred Foot  BOV Best Position  \
Joined                                                     
1998-01-01        80.0          Right   65            GK   
2002-01-01        73.0          Right   70            CB   

            Value(Euro In Millions)  Wage(Euro In Millions)  \
Joined                                                        
1998-01-01                    0.080                   0.001   
2002-01-01                    0.275                   0.003   

            Release Clause(Euro In Millions)  Attacking  Crossing  Finishing  \
Joined                                                                         
1998-01-01                             0.063         87        13         12   
2002-01-01                             0.438        260        58         38   

            Heading Accuracy  Short Passing  Volleys  Skill  Dribbling  Curve  \
Joined                                                                          
1998-01-01                18             26       18     84         14     10   
2002-01-01                70             55       39    232         51     46   

            FK Accuracy  Long Passing  Ball Control  Movement  Acceleration  \
Joined                                                                        
1998-01-01           11            33            16       143            17   
2002-01-01           30            49            56       325            69   

            Sprint Speed  Agility  Reactions  Balance  Power  Shot Power  \
Joined                                                                     
1998-01-01            19       28         47       32    203          48   
2002-01-01            61       54         73       68    316          48   

            Jumping  Stamina  Strength  Long Shots  Mentality  Aggression  \
Joined                                                                      
1998-01-01       52       37        51          15        120          21   
2002-01-01       89       63        75          41        341          71   

            Interceptions  Positioning  Vision  Penalties  Composure  \
Joined                                                                 
1998-01-01             12           10      45         12         20   
2002-01-01             73           55      43         43         56   

            Defending  Marking  Standing Tackle  Sliding Tackle  Goalkeeping  \
Joined                                                                         
1998-01-01         42       17               12              13          331   
2002-01-01        205       70               69              66           53   

            GK Diving  GK Handling  GK Kicking  GK Positioning  GK Reflexes  \
Joined                                                                        
1998-01-01         64           66          64              72           65   
2002-01-01          9         